In [1]:
import geopandas as gpd
import numpy as np

def distribute_excess_population(shapefile_path):
    # 讀取 shapefile
    gdf = gpd.read_file(shapefile_path, encoding='utf-8')  # 添加 encoding 參數來指定編碼方式為 UTF-8

    #指定人數欄位
    people='peo_ac_025'

    # 檢查是否存在 capacity 和 peo_a_025 欄
    if "capacity" not in gdf.columns or people not in gdf.columns:
        raise ValueError("capacity 或 people 欄不存在於 shapefile 中。")

    # 計算 capacity - peo_a_025 欄的值
    gdf["excess"] = gdf["capacity"] - gdf[people]

    # 創建一個欄來存儲分配的人口數
    gdf["allocated_population"] = 0

    # 創建一個字典來存儲防空設施之間的人口移動
    population_movement = {}

    # 創建一個欄來存儲防空洞是否已操作
    gdf["has_movement"] = False

    gdf = gdf.sort_values(by="excess", ascending=True)

    # 迴圈處理每個防空洞
    for index, row in gdf.iterrows():
        excess_population = row["excess"]

        # 檢查當前防空洞的剩餘空間是否大於零且是否已操作過
        if excess_population > 0 or row["has_movement"]:
            continue

        # 找出相鄰的防空洞
        neighbors = gdf[gdf.geometry.intersects(row.geometry)]
        
        # 篩選具有正人數且尚未分配完超額人口的相鄰防空洞
        positive_neighbors = neighbors[
            neighbors["capacity"] - neighbors[people] >= 1
        ]
        if not positive_neighbors.empty:
            total_positive_values = positive_neighbors["capacity"] - positive_neighbors[people]

            # 檢查總權重是否為零，如果為零，則跳過分配
            if total_positive_values.sum() == 0:
                continue

            #若周遭無法負荷
            if total_positive_values.sum()<-excess_population:
                 for neighbor_index, neighbor_row in positive_neighbors.iterrows():
                    extra_population=gdf.at[neighbor_index,"capacity"] -gdf.at[neighbor_index,people] 
                    gdf.at[neighbor_index,people] = gdf.at[neighbor_index,"capacity"] 
                    excess_population+=total_positive_values.sum()
                    gdf.at[neighbor_index, "allocated_population"] += extra_population
                    if index not in population_movement:
                        population_movement[index] = {}
                    population_movement[index][neighbor_index] = extra_population

            else:
            #權重計算
                weight =  -excess_population / total_positive_values.sum()
                for neighbor_index, neighbor_row in positive_neighbors.iterrows():
                    neighbor_population = neighbor_row["capacity"] - neighbor_row[people] 
                    extra_population =  neighbor_population
                    weighted_extra_population = weight * extra_population  # 根據權重計算分配給相鄰防空洞的額外人口
                    new_neighbor_population = gdf.at[neighbor_index, people] + weighted_extra_population
                    new_current_population = gdf.at[index, people] - weighted_extra_population
                    # 更新相鄰防空洞和當前防空洞的人口
                    gdf.at[neighbor_index, people] = new_neighbor_population
                    gdf.at[index, people] = new_current_population
                    # 更新相鄰防空洞的已分配人口數
                    gdf.at[neighbor_index, "allocated_population"] += weighted_extra_population
                    # 更新人口移動的數量
                    if index not in population_movement:
                        population_movement[index] = {}
                    population_movement[index][neighbor_index] = weighted_extra_population

            # 更新當前防空洞的操作狀態為已操作
            gdf.at[index, "has_movement"] = True

    # 儲存變更到新的 shapefile
    output_shapefile = "updated_shapefile.shp"
    gdf.to_file(output_shapefile, encoding='utf-8')
    return population_movement


# 呼叫函數並傳遞 shapefile 的路徑
shapefile_path = "Ho, D - 2023 - NTU Shelter Region II with Population I.shp"
movement_dict = distribute_excess_population(shapefile_path)

# 打印防空設施之間的人口移動情況
gdf = gpd.read_file(shapefile_path, encoding='utf-8')  # 重新讀取 shapefile

for source, targets in movement_dict.items():
    source_name = gdf.loc[source, "name"]  # 獲取源防空洞的名稱
    for target, population in targets.items():
        target_name = gdf.loc[target, "name"]  # 獲取目標防空洞的名稱
        print(f" {source_name} 移動了 {population} 人到 {target_name}")


 博雅教學館 移動了 42.71289076732493 人到 小福樓
 博雅教學館 移動了 87.0596079574437 人到 海洋所
 博雅教學館 移動了 118.56911753989652 人到 思亮館（2）
 博雅教學館 移動了 2954.6583837353346 人到 綜合體育館
 綜合教學館 移動了 136.0 人到 水工所
 農化新館 移動了 250.0 人到 研一舍（2）
 農化新館 移動了 259.0 人到 展書樓
 農化新館 移動了 312.0 人到 農綜大樓
 資工館 移動了 13.137931034482758 人到 漁業科學館
 資工館 移動了 20.278110944527736 人到 心理系南館
 資工館 移動了 103.60401049475263 人到 學新館
 資工館 移動了 437.19321589205396 人到 社會社工館
 資工館 移動了 568.786731634183 人到 社會科學院
 化學館＋積學館 移動了 145.37355049217874 人到 漁業科學館
 化學館＋積學館 移動了 211.00435421589287 人到 生化館
 化學館＋積學館 移動了 243.2849506741082 人到 海洋所
 化學館＋積學館 移動了 331.33714461782023 人到 思亮館（2）
 獸醫館 移動了 23.355994641661084 人到 慶齡工業中心
 獸醫館 移動了 543.6440053583389 人到 明達館
 文學院研究大樓 移動了 140.28710923267508 人到 小福樓
 凝態物理館（2） 移動了 6.073573116030336 人到 全變中心
 凝態物理館（2） 移動了 13.548740028067673 人到 次震宇宙館
 凝態物理館（2） 移動了 1.992853399598575 人到 海洋所
 凝態物理館（2） 移動了 453.3848334563034 人到 綜合體育館
 尊賢館 移動了 20.292428198433424 人到 推廣中心＋城鄉所
 尊賢館 移動了 247.7075718015666 人到 管院一號館
 原分所 移動了 11.64230158808353 人到 漁業科學館
 原分所 移動了 16.898371951866135

C:\Users\USER\AppData\Local\Temp\ipykernel_17664\2113718493.py:86: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shapefile, encoding='utf-8')
